In [1]:
from OSLPPWEIGHTED import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import os
import pandas as pd
from torch.optim.lr_scheduler import StepLR
import pickle

In [2]:
with open('small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
def get_abbrev(source, target): return source[0].upper() + target[0].upper()
def fmeasures(d): return ' '.join([f'{k}={v*100:.2f}' for (k,v) in d.items()])

In [5]:
class FeaturesDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, i): return self.features[i], self.labels[i]

In [6]:
def _create_balanced_sampler(ds:FeaturesDataset):
    freq2 = Counter(ds.labels.tolist())
    class_weight = {x: 1.0 / freq2[x] for x in freq2}
    source_weights = [class_weight[x] for x in ds.labels.tolist()]
    sampler = torch.utils.data.WeightedRandomSampler(source_weights, len(ds.labels.tolist()))
    return sampler

def _create_trn_dataloader(ds, batch_size, balanced):
    if balanced: return torch.utils.data.DataLoader(ds, batch_size=batch_size, sampler=_create_balanced_sampler(ds), drop_last=True)
    else: torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=True)

In [17]:
def train_initial_NN(feats_S, lbls_S, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    assert (lbls_S.unique() == torch.arange(num_src_classes)).all()
    model = nn.Sequential(nn.Linear(params.pca_dim, params.proj_dim), nn.ReLU(), nn.Linear(params.proj_dim, num_src_classes)).cuda().train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     scheduler = StepLR(optimizer, step_size=10, gamma=0.7)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats_S, lbls_S)
    dl = _create_trn_dataloader(ds, 32, balanced)
    for ep in range(num_epochs):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            loss = loss_fn(model(f.cuda()), l.cuda())
            loss.backward()
            
            avg_loss += loss.item()
            optimizer.step()
        print('Loss:', avg_loss / len(dl))
#         scheduler.step()
    return model.eval()

In [18]:
def entropy_loss(logits):
    probs = F.softmax(logits, dim=1)
    entropy =  - probs * probs.log()
    entropy = entropy.sum(dim=1)
    return entropy.mean()

In [19]:
def train_NN(feats, lbls, num_epochs, params, balanced, lr):
    num_src_classes = params.num_common + params.num_src_priv
    # feats, lbls = torch.tensor(feats), torch.tensor(lbls)
    # lbls[lbls == -2] = num_src_classes
    feats, lbls = feats[lbls >= 0], lbls[lbls >= 0]
    feats, lbls = feats[lbls < num_src_classes], lbls[lbls < num_src_classes]
    model = nn.Sequential(nn.Linear(params.pca_dim, params.proj_dim), nn.ReLU(), nn.Linear(params.proj_dim, num_src_classes)).cuda().train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     scheduler = StepLR(optimizer, step_size=10, gamma=0.7)
    loss_fn = nn.CrossEntropyLoss().cuda()
    ds = FeaturesDataset(feats, lbls)
    dl = _create_trn_dataloader(ds, 32, balanced)
    for ep in range(num_epochs):
        avg_loss = 0
        for f,l in dl:
            optimizer.zero_grad()
            loss = loss_fn(model(f.cuda()), l.cuda())
            loss.backward()
            
            avg_loss += loss.item()
            optimizer.step()
        print('Loss:', avg_loss / len(dl))
#         scheduler.step()
    return model.eval()

In [20]:
def predict_NN(model, feats, lbls):
    ds = FeaturesDataset(feats, lbls)
    dl = torch.utils.data.DataLoader(ds, batch_size=32, shuffle=False)
    model = model.eval()
    feats = []
    preds = []
    with torch.no_grad():
        for f,l in dl:
            out = model(f.cuda())
            preds.append(F.softmax(out, dim=1).detach().cpu())
            feats.append(f.detach())
    feats = torch.cat(feats, dim=0)
    preds = torch.cat(preds, dim=0)
    return feats, preds

In [21]:
# sources = ['art', 'clipart', 'product', 'real_world']
# targets = ['art', 'clipart', 'product', 'real_world']

In [22]:
# sources = ['amazon', 'dslr', 'webcam']
# targets = ['amazon', 'dslr', 'webcam']

In [23]:
# sources = ['painting_train', 'real_train', 'sketch_train']
# targets = ['painting_train', 'real_train', 'sketch_train']

In [24]:
# sources = ['train'] 
# targets = ['validation']

In [25]:
def set_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [26]:
results = pd.DataFrame({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': [], 'weights': []})
for (source, target), (common, tgt_private) in config.items():
    for epochs in [10]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.25]:
                for weights in [[0.33, 0.33, 0.33], [0.2, 0.4, 0.4], [0.4, 0.2, 0.4], [0.4, 0.4, 0.2]]:
                    for seed in range(5):
                        set_seed(seed)
                        print(source, '->', target, 'lr=', lr, 'seed=', seed)
                        params = Params(pca_dim=512, proj_dim=128, T=10, n_r=1200, n_r_ratio=None,
                                  dataset='DomainNet_DCC', source=source, target=target,
                                  num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                        (feats_S, lbls_S), (feats_T, lbls_T) = create_datasets_sub(params.dataset, 
                                                                                   params.source, 
                                                                                   params.target, 
                                                                                   common, 
                                                                                   tgt_private)
                        params.n_r = int(len(lbls_T) * n_r)
                        num_src_classes = params.num_common + params.num_src_priv

                        # l2 normalization and pca
                        feats_S, feats_T = do_l2_normalization(feats_S, feats_T)
                        feats_S, feats_T = do_pca(feats_S, feats_T, params.pca_dim)
                        feats_S, feats_T = do_l2_normalization(feats_S, feats_T)

                        # initial
                        feats_S, feats_T = torch.tensor(feats_S), torch.tensor(feats_T)
                        lbls_S, lbls_T = torch.tensor(lbls_S), torch.tensor(lbls_T)
                        feats_all = torch.cat((feats_S, feats_T), dim=0)
                        pseudo_labels = -torch.ones_like(lbls_T)
                        rejected = torch.zeros_like(pseudo_labels)

                        t = 1

                        model = train_initial_NN(feats_S, lbls_S, epochs, params, balanced=True, lr=lr)
                        feats_S_2, preds_S_2 = predict_NN(model, feats_S, lbls_S)
                        feats_T_2, preds_T_2 = predict_NN(model, feats_T, lbls_T)
                        print((feats_S_2 == feats_S).all(), (feats_T_2 == feats_T).all())
                        assert (feats_S_2 == feats_S).all() and (feats_T_2 == feats_T).all()

                        confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                        selected = select_closed_set_pseudo_labels_weighted(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, weights)
                        selected = torch.tensor(selected)
                        rejected = select_initial_rejected_weighted(confs, params.n_r, preds_T_2, weights)

                        rejected = torch.tensor(rejected)
                        selected = selected * (1 - rejected)

                        pseudo_labels = cs_pseudo_labels.clone()
                        pseudo_labels[rejected == 1] = num_src_classes
                        pseudo_labels[(rejected == 0) * (selected == 0)] = -1

                        metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, 
                                lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                        where = torch.where((selected == 1) + (rejected == 1))[0]
                        metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                        print('______')
                        print(f'Iteration t={t}')
                        print('all: ', fmeasures(metrics))
                        print('selected: ', fmeasures(metrics_selected))

                        for t in range(2, params.T):
                            model = train_NN(feats_all, torch.cat((lbls_S, pseudo_labels), axis=0), epochs, params, balanced=True, lr=lr)
                            feats_S_2, preds_S_2 = predict_NN(model, feats_S, lbls_S)
                            feats_T_2, preds_T_2 = predict_NN(model, feats_T, lbls_T)
                            assert (feats_S_2 == feats_S).all() and (feats_T_2 == feats_T).all()

                            confs, cs_pseudo_labels = preds_T_2.max(dim=1)
                            selected = torch.tensor(select_closed_set_pseudo_labels_weighted(cs_pseudo_labels.numpy(), confs.numpy(), t, params.T, preds_T_2, weights))
                            selected = selected * (1 - rejected)

                            scores = get_weighted_scores(confs, preds_T_2, weights)

                            rejected_new = (scores < min(scores[rejected == 1].max().item(), scores[selected == 1].min().item())).int()



                            rejected[rejected_new == 1] = 1
                            selected = selected * (1 - rejected)

                            pseudo_labels = cs_pseudo_labels.clone()
                            pseudo_labels[rejected == 1] = num_src_classes
                            pseudo_labels[(rejected == 0) * (selected == 0)] = -1

                            metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), rejected.numpy())
                            where = torch.where((selected == 1) + (rejected == 1))[0]
                            metrics_selected = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T[where].numpy(), cs_pseudo_labels[where].numpy(), rejected[where].numpy())
                            print('______')
                            print(f'Iteration t={t}')
                            print('all: ', fmeasures(metrics))
                            print('selected: ', fmeasures(metrics_selected))

                        print((pseudo_labels == -1).sum())

                        _rejected = rejected
                        metrics = evaluate_T(params.num_common, params.num_src_priv, params.num_tgt_priv, lbls_T.numpy(), cs_pseudo_labels.numpy(), _rejected.numpy())
                        print('all: ', fmeasures(metrics))

                        results = results.append({'weights': str(weights), 'source': source, 'target': target, 'desc': fmeasures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs}, ignore_index=True)

painting -> sketch lr= 0.001 seed= 0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Loss: 2.562207261721293
Loss: 1.2393408083411122
Loss: 0.7387420686464461
Loss: 0.564478018454143
Loss: 0.5006312367619661
Loss: 0.4301535349201273
Loss: 0.38650771423622415
Loss: 0.35652905312322436
Loss: 0.32989208191317854
Loss: 0.2757383891277843
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=52.23 cs/acc_c=53.77 os/recall_knw=95.06 os/recall_unk=18.72 total/acc_i=39.30 total/acc_c=51.13 total/h_score=27.63
selected:  cs/acc_i=63.07 cs/acc_c=64.16 os/recall_knw=76.05 os/recall_unk=82.66 total/acc_i=67.53 total/acc_c=60.48 total/h_score=69.11
Loss: 2.474956455183964
Loss: 1.0758645984472013
Loss: 0.6366970372550628
Loss: 0.5238751575496852
Loss: 0.431535816952294
Loss: 0.3821670485331732
Loss: 0.3484044816052797
Loss: 0.2945105408263557
Loss: 0.278670582208125
Loss: 0.25479272973961103
______
Iteration t=2


Loss: 0.4054354298016662
Loss: 0.35515543242050296
Loss: 0.3062824573475147
Loss: 0.2746727056648015
Loss: 0.2497996680223893
Loss: 0.21698428258115862
Loss: 0.2058340051046615
______
Iteration t=6
all:  cs/acc_i=51.02 cs/acc_c=52.72 os/recall_knw=55.09 os/recall_unk=75.07 total/acc_i=50.12 total/acc_c=38.84 total/h_score=49.60
selected:  cs/acc_i=50.87 cs/acc_c=52.67 os/recall_knw=54.86 os/recall_unk=75.31 total/acc_i=50.08 total/acc_c=38.73 total/h_score=49.53
Loss: 2.3243443549163945
Loss: 0.8537590163267725
Loss: 0.5044855126520482
Loss: 0.4155593864256289
Loss: 0.3402286485988435
Loss: 0.29660773213680197
Loss: 0.2571634112637702
Loss: 0.24337227739454284
Loss: 0.22500657909587632
Loss: 0.19784034023863997
______
Iteration t=7
all:  cs/acc_i=51.13 cs/acc_c=52.77 os/recall_knw=55.09 os/recall_unk=75.13 total/acc_i=50.14 total/acc_c=38.85 total/h_score=49.61
selected:  cs/acc_i=51.11 cs/acc_c=52.76 os/recall_knw=55.08 os/recall_unk=75.13 total/acc_i=50.13 total/acc_c=38.83 total/h_s

Loss: 1.053973794275639
Loss: 0.6383561838199111
Loss: 0.4825239023741554
Loss: 0.41764359716691224
Loss: 0.36435126129756956
Loss: 0.3376537155652163
Loss: 0.31094812594500243
Loss: 0.2907656703655626
Loss: 0.26059987828792897
______
Iteration t=2
all:  cs/acc_i=51.09 cs/acc_c=52.55 os/recall_knw=65.92 os/recall_unk=64.01 total/acc_i=49.11 total/acc_c=43.01 total/h_score=50.69
selected:  cs/acc_i=49.01 cs/acc_c=50.64 os/recall_knw=46.03 os/recall_unk=85.27 total/acc_i=53.83 total/acc_c=36.18 total/h_score=48.33
Loss: 2.419768670830158
Loss: 0.9737176080362513
Loss: 0.5894018302953571
Loss: 0.4795452101378266
Loss: 0.39787495635244824
Loss: 0.3366119257012092
Loss: 0.3030413058031043
Loss: 0.2655369353335385
Loss: 0.24452268607367617
Loss: 0.23520932449113338
______
Iteration t=3
all:  cs/acc_i=50.91 cs/acc_c=52.58 os/recall_knw=61.77 os/recall_unk=67.47 total/acc_i=49.21 total/acc_c=41.36 total/h_score=50.27
selected:  cs/acc_i=49.43 cs/acc_c=51.59 os/recall_knw=51.72 os/recall_unk=79

Loss: 2.30148971270755
Loss: 0.8292469562762287
Loss: 0.5240442064178892
Loss: 0.38935063286606536
Loss: 0.32923208663900533
Loss: 0.2788081255507659
Loss: 0.26206825461342514
Loss: 0.22687526519198817
Loss: 0.22941539725993734
Loss: 0.1921249508145321
______
Iteration t=8
all:  cs/acc_i=50.45 cs/acc_c=52.14 os/recall_knw=57.58 os/recall_unk=70.03 total/acc_i=48.76 total/acc_c=39.32 total/h_score=49.08
selected:  cs/acc_i=50.45 cs/acc_c=52.14 os/recall_knw=57.58 os/recall_unk=70.03 total/acc_i=48.76 total/acc_c=39.32 total/h_score=49.08
Loss: 2.301852303434653
Loss: 0.8259137815450767
Loss: 0.4998835156638309
Loss: 0.4021817652055942
Loss: 0.3268264917382206
Loss: 0.2970372977187909
Loss: 0.27160371284917056
Loss: 0.21796611846383823
Loss: 0.20975740963363076
Loss: 0.19640479555939774
______
Iteration t=9
all:  cs/acc_i=51.06 cs/acc_c=52.63 os/recall_knw=57.58 os/recall_unk=70.03 total/acc_i=48.76 total/acc_c=39.32 total/h_score=49.08
selected:  cs/acc_i=51.06 cs/acc_c=52.63 os/recall_

Loss: 2.3604359270690325
Loss: 0.8931303446169023
Loss: 0.5361167096988463
Loss: 0.44004737927671117
Loss: 0.35809902308178154
Loss: 0.3172650226589405
Loss: 0.2831554764083454
Loss: 0.263930600571942
Loss: 0.23675711993208695
Loss: 0.20705559530428477
______
Iteration t=4
all:  cs/acc_i=49.89 cs/acc_c=51.46 os/recall_knw=55.81 os/recall_unk=73.89 total/acc_i=50.05 total/acc_c=39.36 total/h_score=49.87
selected:  cs/acc_i=47.47 cs/acc_c=49.41 os/recall_knw=51.87 os/recall_unk=77.01 total/acc_i=49.35 total/acc_c=36.52 total/h_score=47.65
Loss: 2.3448159330559575
Loss: 0.8845957248280737
Loss: 0.5223249349135235
Loss: 0.4009253408245462
Loss: 0.3409267184499916
Loss: 0.31828785534932524
Loss: 0.25917279904232865
Loss: 0.2672267162930766
Loss: 0.2292583823889868
Loss: 0.20797931772333308
______
Iteration t=5
all:  cs/acc_i=49.89 cs/acc_c=51.39 os/recall_knw=55.74 os/recall_unk=73.95 total/acc_i=50.05 total/acc_c=39.33 total/h_score=49.85
selected:  cs/acc_i=48.40 cs/acc_c=50.31 os/recall_

...

Loss: 0.5878066225472279
Loss: 0.5255038177477565
______
Iteration t=2
all:  cs/acc_i=67.12 cs/acc_c=67.23 os/recall_knw=64.66 os/recall_unk=69.93 total/acc_i=59.82 total/acc_c=56.18 total/h_score=61.88
selected:  cs/acc_i=61.48 cs/acc_c=62.59 os/recall_knw=46.47 os/recall_unk=94.61 total/acc_i=60.97 total/acc_c=47.36 total/h_score=60.98
Loss: 2.578081954067404
Loss: 1.3708654804663225
Loss: 0.9387236702171239
Loss: 0.8095121966166929
Loss: 0.7059829233722253
Loss: 0.6734464297917756
Loss: 0.6038542936471376
Loss: 0.5627989426932551
Loss: 0.55102877183394
Loss: 0.506673989309506
______
Iteration t=3
all:  cs/acc_i=66.73 cs/acc_c=66.95 os/recall_knw=59.68 os/recall_unk=77.57 total/acc_i=60.52 total/acc_c=54.11 total/h_score=62.93
selected:  cs/acc_i=63.49 cs/acc_c=64.70 os/recall_knw=50.69 os/recall_unk=90.07 total/acc_i=60.66 total/acc_c=49.75 total/h_score=62.40
Loss: 2.543487993530605
Loss: 1.3100071005199267
Loss: 0.8809536415597666
Loss: 0.7589851418267126
Loss: 0.6643690531668456


Loss: 2.481375641939117
Loss: 1.2120185250916131
Loss: 0.8731727620692757
Loss: 0.7127244724490778
Loss: 0.6380603378865777
Loss: 0.5638091393360277
Loss: 0.5343390340969815
Loss: 0.4987383917943249
Loss: 0.4682611560191565
Loss: 0.4352555463347978
______
Iteration t=8
all:  cs/acc_i=67.91 cs/acc_c=68.12 os/recall_knw=56.31 os/recall_unk=79.90 total/acc_i=59.53 total/acc_c=52.09 total/h_score=62.03
selected:  cs/acc_i=67.91 cs/acc_c=68.12 os/recall_knw=56.31 os/recall_unk=79.90 total/acc_i=59.53 total/acc_c=52.09 total/h_score=62.03
Loss: 2.48270826271879
Loss: 1.2310545044701273
Loss: 0.8499781169542452
Loss: 0.7092316527434481
Loss: 0.6270047402599963
Loss: 0.5664213852184575
Loss: 0.5510018781675556
Loss: 0.4949737236933495
Loss: 0.47575753578931335
Loss: 0.44949637177756163
______
Iteration t=9
all:  cs/acc_i=67.79 cs/acc_c=68.12 os/recall_knw=56.31 os/recall_unk=79.90 total/acc_i=59.53 total/acc_c=52.09 total/h_score=62.03
selected:  cs/acc_i=67.79 cs/acc_c=68.12 os/recall_knw=56.

Loss: 2.561079726074681
Loss: 1.2837187103378824
Loss: 0.9063423782974095
Loss: 0.7509966779064823
Loss: 0.667102397262276
Loss: 0.6167773975954427
Loss: 0.5992457626056877
Loss: 0.5190226067325253
Loss: 0.4831751099912635
Loss: 0.46244022082456776
______
Iteration t=4
all:  cs/acc_i=67.83 cs/acc_c=67.91 os/recall_knw=56.54 os/recall_unk=81.06 total/acc_i=60.33 total/acc_c=52.63 total/h_score=62.77
selected:  cs/acc_i=66.68 cs/acc_c=67.22 os/recall_knw=53.58 os/recall_unk=85.46 total/acc_i=60.27 total/acc_c=51.11 total/h_score=62.60
Loss: 2.5150367216086287
Loss: 1.2391736567269809
Loss: 0.8783539503688094
Loss: 0.7279385156212491
Loss: 0.6516952899335319
Loss: 0.5722798176390357
Loss: 0.5559431855648631
Loss: 0.49844749733244526
Loss: 0.4784858753616341
Loss: 0.4522155043532659
______
Iteration t=5
all:  cs/acc_i=67.87 cs/acc_c=68.03 os/recall_knw=56.43 os/recall_unk=81.40 total/acc_i=60.38 total/acc_c=52.58 total/h_score=62.81
selected:  cs/acc_i=67.40 cs/acc_c=67.74 os/recall_knw=55

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Loss: 2.680974315122231
Loss: 1.5903894206912248
Loss: 1.137612421795265
Loss: 0.9610049062475716
Loss: 0.8090056801579663
Loss: 0.7600589174892485
Loss: 0.6991563686697754
Loss: 0.6555561146785304
Loss: 0.6472147124790654
Loss: 0.597991328878501
tensor(True) tensor(True)
______
Iteration t=1
all:  cs/acc_i=65.01 cs/acc_c=65.67 os/recall_knw=81.78 os/recall_unk=39.37 total/acc_i=53.75 total/acc_c=60.19 total/h_score=47.92
selected:  cs/acc_i=57.67 cs/acc_c=58.81 os/recall_knw=46.80 os/recall_unk=96.73 total/acc_i=63.34 total/acc_c=49.26 total/h_score=63.16
Loss: 2.6186220035460837
Loss: 1.4773788112373167
Loss: 1.0286256319658769
Loss: 0.8600234164707903
Loss: 0.7764996628542453
Loss: 0.7096467306072586
Loss: 0.6530070490595223
Loss: 0.6321680951809537
Loss: 0.5894582578957369
Loss: 0.528064637368428
______
Iteration t=2
all:  cs/acc_i=66.97 cs/acc_c=67.34 os/r

Loss: 0.49749425686135584
Loss: 0.4564200506830702
Loss: 0.44537140179653556
______
Iteration t=6
all:  cs/acc_i=67.32 cs/acc_c=67.70 os/recall_knw=58.62 os/recall_unk=77.66 total/acc_i=60.01 total/acc_c=53.55 total/h_score=62.54
selected:  cs/acc_i=67.18 cs/acc_c=67.61 os/recall_knw=58.24 os/recall_unk=78.37 total/acc_i=60.05 total/acc_c=53.38 total/h_score=62.62
Loss: 2.484122005681838
Loss: 1.2115757782613077
Loss: 0.8488386004201828
Loss: 0.6948819572646772
Loss: 0.6358779590697058
Loss: 0.5772925560633021
Loss: 0.5392591575340878
Loss: 0.4850277528166771
Loss: 0.4642150536660225
Loss: 0.4444373811204587
______
Iteration t=7
all:  cs/acc_i=68.61 cs/acc_c=69.05 os/recall_knw=58.54 os/recall_unk=77.74 total/acc_i=60.01 total/acc_c=53.52 total/h_score=62.54
selected:  cs/acc_i=68.61 cs/acc_c=69.05 os/recall_knw=58.54 os/recall_unk=77.87 total/acc_i=60.04 total/acc_c=53.53 total/h_score=62.58
Loss: 2.4780596499462204
Loss: 1.2115030020595077
Loss: 0.8561366202840843
Loss: 0.71408861492

In [9]:
results.to_csv('results_weighted_scores/dcc__conf__small__NEW__weighted__nn_raw.csv', header=True, index=False)

In [20]:
# df = pd.read_csv('office_home_oslpp_nn_raw.csv')
# df['h_score'] = df['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))
# import matplotlib.pyplot as plt
# for n, gr in df.groupby(['source', 'target']):
#     stat = gr.groupby(['lr', 'n_r'])['h_score'].mean()
#     exps = np.array(list(stat.index))
#     fig, ax = plt.subplots()
#     ax.scatter(exps[:, 0], exps[:, 1], c=stat.values)
#     for i, txt in enumerate(stat.values):
#         ax.annotate(round(txt, 2), (exps[i][0], exps[i][1]))
#     plt.title(n)
#     plt.xlabel('lr')
#     plt.ylabel('n_r')
#     plt.xticks(exps[:, 0])
#     plt.yticks(exps[:, 1])
#     fig.colorbar(ax.get_children()[0])
#     plt.show()

In [27]:
results

,source,target,desc,lr,seed,epochs,weights,n_r
0,painting,sketch,cs/acc_i=50.87 cs/acc_c=52.71 os/recall_knw=58...,0.001,0.0,10.0,"[0.33, 0.33, 0.33]",0.10
1,painting,sketch,cs/acc_i=51.02 cs/acc_c=52.77 os/recall_knw=55...,0.001,1.0,10.0,"[0.33, 0.33, 0.33]",0.10
2,painting,sketch,cs/acc_i=50.38 cs/acc_c=51.95 os/recall_knw=59...,0.001,2.0,10.0,"[0.33, 0.33, 0.33]",0.10
3,painting,sketch,cs/acc_i=51.70 cs/acc_c=53.25 os/recall_knw=59...,0.001,3.0,10.0,"[0.33, 0.33, 0.33]",0.10
4,painting,sketch,cs/acc_i=51.06 cs/acc_c=52.63 os/recall_knw=57...,0.001,4.0,10.0,"[0.33, 0.33, 0.33]",0.10
...,...,...,...,...,...,...,...,...
235,sketch,painting,cs/acc_i=68.65 cs/acc_c=69.12 os/recall_knw=56...,0.001,0.0,10.0,"[0.4, 0.4, 0.2]",0.25
236,sketch,painting,cs/acc_i=68.03 cs/acc_c=68.56 os/recall_knw=54...,0.001,1.0,10.0,"[0.4, 0.4, 0.2]",0.25
237,sketch,painting,cs/acc_i=68.73 cs/acc_c=69.14 os/recall_knw=57...,0.001,2.0,10.0,"[0.4, 0.4, 0.2]",0.25
238,sketch,painting,cs/acc_i=68.42 cs/acc_c=69.02 os/recall_knw=58...,0.001,3.0,10.0,"[0.4, 0.4, 0.2]",0.25


In [30]:
results['epoch'] = pd.Series(results.index).apply(lambda i: epochs[(i // 9) % 3])

In [10]:
results = pd.read_csv('results_weighted_scores/dcc__conf__small__NEW__weighted__nn_raw.csv')

In [11]:
wd = results.copy()
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [12]:
wd

,source,target,desc,lr,seed,epochs,weights,n_r,h_score
0,painting,sketch,cs/acc_i=50.23 cs/acc_c=51.69 os/recall_knw=56...,0.001,0.0,10.0,"[0.33, 0.33, 0.33]",0.10,48.68
1,painting,sketch,cs/acc_i=52.11 cs/acc_c=53.79 os/recall_knw=58...,0.001,1.0,10.0,"[0.33, 0.33, 0.33]",0.10,50.19
2,painting,sketch,cs/acc_i=51.06 cs/acc_c=52.70 os/recall_knw=58...,0.001,2.0,10.0,"[0.33, 0.33, 0.33]",0.10,49.43
3,painting,sketch,cs/acc_i=50.15 cs/acc_c=51.65 os/recall_knw=57...,0.001,3.0,10.0,"[0.33, 0.33, 0.33]",0.10,49.93
4,painting,sketch,cs/acc_i=50.79 cs/acc_c=52.41 os/recall_knw=56...,0.001,4.0,10.0,"[0.33, 0.33, 0.33]",0.10,49.27
...,...,...,...,...,...,...,...,...,...
235,sketch,painting,cs/acc_i=68.61 cs/acc_c=68.97 os/recall_knw=58...,0.001,0.0,10.0,"[0.4, 0.4, 0.2]",0.25,63.69
236,sketch,painting,cs/acc_i=68.69 cs/acc_c=68.92 os/recall_knw=53...,0.001,1.0,10.0,"[0.4, 0.4, 0.2]",0.25,62.27
237,sketch,painting,cs/acc_i=69.12 cs/acc_c=69.16 os/recall_knw=59...,0.001,2.0,10.0,"[0.4, 0.4, 0.2]",0.25,62.99
238,sketch,painting,cs/acc_i=67.63 cs/acc_c=67.95 os/recall_knw=55...,0.001,3.0,10.0,"[0.4, 0.4, 0.2]",0.25,61.61


In [21]:
wd.groupby(['source', 'target', 'lr', 'n_r', 'epochs', 'weights'])['h_score'].mean().to_frame(name = 'h_score').reset_index().sort_values(by='h_score', ascending=False).drop_duplicates(['source', 'target'])

,source,target,lr,n_r,epochs,weights,h_score
41,sketch,real,0.001,0.1,10.0,"[0.33, 0.33, 0.33]",80.042
2,painting,real,0.001,0.1,10.0,"[0.4, 0.2, 0.4]",78.174
18,real,painting,0.001,0.1,10.0,"[0.4, 0.2, 0.4]",64.244
35,sketch,painting,0.001,0.1,10.0,"[0.4, 0.4, 0.2]",62.856
27,real,sketch,0.001,0.1,10.0,"[0.4, 0.4, 0.2]",55.742
8,painting,sketch,0.001,0.1,10.0,"[0.2, 0.4, 0.4]",49.892


In [14]:
wd['name'] = wd['source'].apply(lambda x: x[0].upper()) + wd['target'].apply(lambda x: x[0].upper())

In [15]:
wd.pivot_table(index=['n_r', 'lr', 'epochs', 'weights'], columns='name', values='h_score', aggfunc=[lambda x: round(np.mean(x), 2)]).sort_index(key=lambda x: x, ascending=False)

<lambda>                              \
name                                       PR     PS     RP     RS     SP   
n_r  lr    epochs weights                                                   
0.25 0.001 10.0   [0.4, 0.4, 0.2]       77.12  49.62  64.09  54.61  62.66   
                  [0.4, 0.2, 0.4]       77.31  49.79  64.19  54.69  62.37   
                  [0.33, 0.33, 0.33]    77.46  49.25  63.90  55.06  62.69   
                  [0.2, 0.4, 0.4]       77.27  49.43  63.96  54.76  62.39   
0.10 0.001 10.0   [0.4, 0.4, 0.2]       78.10  49.70  63.82  55.74  62.86   
                  [0.4, 0.2, 0.4]       78.17  49.38  64.24  55.57  62.60   
                  [0.33, 0.33, 0.33]    78.03  49.50  63.74  55.45  62.65   
                  [0.2, 0.4, 0.4]       78.03  49.89  63.98  55.41  62.68   

                                             
name                                     SR  
n_r  lr    epochs weights                    
0.25 0.001 10.0   [0.4, 0.4, 0.2]     77.54  
                  [0.4, 0.2, 0.4]     77.29  
                  [0.33, 0.33, 0.33]  78.31  
                  [0.2, 0.4, 0.4]     78.08  
0.10 0.001 10.0   [0.4, 0.4, 0.2]     80.03  
                  [0.4, 0.2, 0.4]     79.44  
                  [0.33, 0.33, 0.33]  80.04  
                  [0.2, 0.4, 0.4]     79.97

In [ ]:
wd.set_index(['lr', 'epoch', 'n_r']).head(30)

In [44]:
wd = pd.read_csv('results/dcc__entropy__StepLR__50_epochs__nn_raw.csv')

In [45]:
wd['h_score'] = wd['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [46]:
wd

,source,target,desc,lr,seed,n_r,h_score
0,painting_train,real_train,cs/acc_i=59.98 cs/acc_c=60.09 os/recall_knw=71...,0.0005,0.0,0.15,54.40
1,painting_train,sketch_train,cs/acc_i=26.40 cs/acc_c=25.41 os/recall_knw=53...,0.0005,0.0,0.15,30.42
2,real_train,painting_train,cs/acc_i=39.74 cs/acc_c=39.39 os/recall_knw=65...,0.0005,0.0,0.15,40.83
3,real_train,sketch_train,cs/acc_i=24.48 cs/acc_c=23.68 os/recall_knw=51...,0.0005,0.0,0.15,29.29
4,sketch_train,painting_train,cs/acc_i=37.51 cs/acc_c=34.45 os/recall_knw=70...,0.0005,0.0,0.15,35.06
5,sketch_train,real_train,cs/acc_i=56.05 cs/acc_c=57.00 os/recall_knw=71...,0.0005,0.0,0.15,48.56
